In [ ]:
# Imports
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

import pandas
from pandas_datareader import data as pdr
import yfinance as yfin
import datetime

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

from google.colab import files

import shutil

import time

import os

from pickle import TRUE

import matplotlib.pyplot as plt
plt.gcf().set_size_inches(22, 15, forward=True)

# Monte o Google Drive no ambiente do Colab


<Figure size 2200x1500 with 0 Axes>

In [ ]:
# Defining our Deep Q-Learning Trader

class AI_Trader():

# -----------------------------------------------------------------------

  # CONSTRUTOR

  def __init__(self, state_size, action_space=3, model_name="AITrader"):

    self.state_size = state_size # Tamanho da entrada da rede neural
    self.action_space = action_space # Espaço de ação será 3, Comprar, Vender, Sem Ação (Tamanho da saída da rede neural)
    self.memory = deque(maxlen=2000) # Memória com 2000 posições. A função Deque permite adicionar elementos ao final, enquanto remove elementos do início.
    self.inventory = [] # Terá as comprar que já fizemos
    self.model_name = model_name # Nome do modelo para o Keras

    self.model = self.model_builder() # Inicializa um modelo e de rede neural e salva na classe

# -----------------------------------------------------------------------

  # DEFININDO A REDE NEURAL

  def model_builder(self):

    model = tf.keras.models.Sequential()
    model.add(layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(layers.Dense(units=64, activation='relu'))
    model.add(layers.Dense(units=128, activation='relu'))
    model.add(layers.Dense(units=self.action_space, activation='linear')) # De maneira geral, teremos 3 saída na rede geral (número de espaços de ação)


    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001)); # Compilamos o modelo

    return model # Retornamos o modelo pela função.

# -----------------------------------------------------------------------

  # FUNÇÃO DE TRADE
  # Usa o Epsilon e um número aleatório para definir se usará um dado aleatório ou a previsão da rede.

  def trade(self, state):
    actions = self.model.predict(state)
    return np.argmax(actions[0])  # Sem aleatoriedade ou com aleatoriedade controlada pelo parâmetro
# -----------------------------------------------------------------------

In [ ]:
# Stock Market Data Preprocessing

# Definiremos algumas funções auxiliares

# Sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# Função para formatar texto
def stock_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

# Busca dados no Yahoo Finance
# Formato data = "yyyy-mm-dd"
def dataset_loader(stock_name, initial_date, final_date):

  yfin.pdr_override()

  dataset = pdr.get_data_yahoo(stock_name, start=initial_date, end=final_date)

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[1]).split()[0]

  close = dataset['Close']

  return close

In [ ]:
# State Creator


def state_creator(data, timestep, window_size):

  # O index incial (starting_id) será o timestep (passos/dias que já foram dados)
  # menos o tamanho da janela, que serão os dias olhados para trás.
  starting_id = timestep - window_size + 1


  # Lógica para preencher os dados vindos da tabela Data, no array windowed_data

  if starting_id >= 0: # No geral este será a condição sempre executada
    windowed_data = data[starting_id: timestep + 1]

  else: # Condição executada apenas nos primeiros valores
    windowed_data =- starting_id * [data[0]] + list(data[0:timestep + 1])

  state = [] # Criação de uma array para retorno, com o estado.

  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))

  return np.array([state])

In [ ]:
def get_buy_and_hold(data):
  first_price = data.iloc[0]
  last_price = data.iloc[-1]
  return ((last_price-first_price)/first_price)*100

def get_drawdown(data):
  data['HighValue'] = data['total_profit'].cummax()
  data['Drawdown'] = data['total_profit']-data['HighValue']
  return data['Drawdown'].min(),data['Drawdown'].mean()

def run_trained_model(model_path, data, window_size):
    trader = AI_Trader(window_size)
    trader.model = tf.keras.models.load_model(model_path)

    state = state_creator(data, 0, window_size + 1)
    quant_operacoes=0
    operacoes_lucro=0
    operacoes_prejuizo=0
    total_profit = 0
    ganho=0
    perda=0
    trader.inventory.clear()
    trader.inventory = []
    gt = {'total_profit': []}

    for t in tqdm(range(len(data) - 1)):
        action = trader.trade(state)
        next_state = state_creator(data, t + 1, window_size + 1)

        # Sem ação
        if action == 0:
          # Apenas um print e Recompensa = 0
          print(" - Sem ação | Total de papeis no portfolio = ", len(trader.inventory))

        # Compra
        if action == 1:
          # Recompensa = 0

          # Adicionamos a ação comprada na array de portfolio
          trader.inventory.append(data[t])

          print(" - AI Trader Comprou: ", stock_price_format(data[t]))

        # Venda (Deve possuir ações no portfolio)
        elif action == 2 and len(trader.inventory) > 0:

          quant_operacoes+=1

          # Remove última ação do portfólio e a retorna
          buy_price = trader.inventory.pop(0)

          #total_profit += data[t] - buy_price # Soma ao lucro/prejuízo total

          if (data[t] - buy_price > 0):
            operacoes_lucro+=1
            ganho+=((data[t]-buy_price)/buy_price)*100
            total_profit +=((data[t]-buy_price)/buy_price)*100
          else:
            operacoes_prejuizo+=1
            perda+=((data[t]-buy_price)/buy_price)*100
            total_profit +=((data[t]-buy_price)/buy_price)*100

          gt['total_profit'].append(total_profit)

          print(" - AI Trader Vendeu: ", stock_price_format(data[t]), " - Lucro: " + stock_price_format(data[t] - buy_price) )
        state = next_state


    taxa_acerto=(operacoes_lucro/quant_operacoes)*100
    taxa_erro=(operacoes_prejuizo/quant_operacoes)*100

    plt.figure()
    plt.plot(gt['total_profit'],linewidth=1)
    plt.xlabel('Operações')
    plt.ylabel('(%)')
    plt.suptitle(f"Resultado {total_profit:.2f}%")
    plt.title("BBAS3.SA, Periodo de Avaliação: 01-01-2023 a 01-06-2023", fontsize=10)
    plt.legend(['Capital'])
    plt.tight_layout()
    plot_filename = 'resultado_plot.png'
    plt.savefig(plot_filename)
    #files.download(plot_filename)
    aux = pd.DataFrame(gt)
    minD,meanD = get_drawdown(aux)
    print("Min = ",minD)
    print("Mean = ",meanD)

    return total_profit, taxa_acerto, taxa_erro, ganho, perda, quant_operacoes


uploaded = files.upload()
file_name = list(uploaded.keys())[0]
model_path = file_name

new_data = dataset_loader("BBAS3.SA", "2023-01-01", "2023-06-01")
inicio = time.time()

total_profit, taxa_acerto, taxa_erro, ganho, perda, quant_operacoes= run_trained_model(model_path, new_data, window_size = 10)

fim = time.time()

tempo_execucao=fim - inicio
buy_and_hold = get_buy_and_hold(new_data)
#Print do tempo que demorou para rodar a parte específica do código
print(f"Tempo de Execucao: {tempo_execucao} segundos")
print(f"Taxa de Acerto: {taxa_acerto} %")
print(f"Taxa de Erro: {taxa_erro} %")
print(f"Ganho: {ganho} %")
print(f"Perda: {abs(perda)} %")
print(f"payoff: {ganho/abs(perda)}")
print(f"Lucro Total: {total_profit} %")
print(f"Buy and Hold: {buy_and_hold} %")
print(f"Total de Operacoes: {quant_operacoes}")
#vale3 petr4 itub4 bbdc4 bbas3, fonte:trading map